<a href="https://colab.research.google.com/github/rebecabarroso/econometria_aplicada/blob/main/ANALISE_ECONOMETRIA_APLICADA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise de dados em Painel: Modelos Estáticos e Dinâmicos**

## Instalando e importando pacotes que virão a ser utilizados na análise.

In [ ]:
# Instalando pacotes.
!pip install matplotlib
!pip install pandas statsmodels linearmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.2 MB/s eta 0:00:00


In [ ]:
# Chamando bibliotecas.

# Pandas realiza alterações e ajustes direto na base da dados.
import pandas as pd

# Seaborn e matplotlib serão utilizados para gerar gráficos de análise.
import seaborn as sns
import matplotlib.pyplot as plt

# Numpy é um pacote de operações matemáticas.
import numpy as np

# Linearmodels traz as soluções para realizar as análises de dados em painel.
from linearmodels.panel import PanelOLS, RandomEffects, compare
from linearmodels.iv import IV2SLS

In [ ]:
# Ativando a ligação com o google drive para subir a base mais facilmente.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Definindo os ajustes iniciais de preparo da base.

In [ ]:
# Definindo a base de dados "dados" com o arquivo xlsx.
dados = pd.read_excel('/content/base de dados.xlsx')

In [ ]:
# Removendo valores nulos, afim de evitar problemas com os dados em painel.
dados.dropna(inplace=True)

# Describe nos dados para análise descritiva.
dados.describe()

,cod,ano,maes_fam,analf,theil,gini,idhm,idhm_educ,ensino_medio,renda
count,204.00000,204.0000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000
mean,51.50000,2005.0000,79.090637,40.712941,0.539020,0.558480,0.477426,0.322966,11.769608,202.603627
std,29.51588,5.0123,11.105367,8.835575,0.125792,0.066163,0.097580,0.131892,6.625598,86.026286
min,1.00000,2000.0000,45.620000,12.510000,0.250000,0.410000,0.281000,0.090000,1.290000,69.200000
25%,26.00000,2000.0000,70.707500,34.377500,0.450000,0.510000,0.384750,0.196000,6.652500,140.020000
50%,51.50000,2005.0000,79.435000,41.255000,0.520000,0.550000,0.504500,0.347500,10.375000,194.840000
75%,77.00000,2010.0000,88.182500,47.005000,0.610000,0.600000,0.563250,0.434250,15.797500,241.447500
max,102.00000,2010.0000,100.000000,57.790000,1.240000,0.820000,0.721000,0.635000,43.160000,792.540000


In [ ]:
# Para as análises de dados em painel, precisamos definir o index as duas variáveis que serão analisadas ao longo do tempo.
# Basicamente, as variáveis não estarão mais presentes na base como variável.
dados.set_index(['mun', 'ano'], inplace=True)

# **Modelos Estáticos**
### Capturam a relação contemporânea entre as variáveis, assumindo que os efeitos das variáveis explicativas sobre a variável dependente são instantâneos e não se alteram ao longo do tempo.

## Modelo de efeitos fixos
### Observa todas as diferenças não observadas e constantes de cada unidade individual ao longo do tempo.

In [ ]:
# mod_ef define a chamada do modelo de efeitos fixos, utilizando a classe PanelOLS;
# "from_formula" é um método utilizado para especificar o modelo usando uma fórmula no estilo patsy (mais conveniente);
# a primeira variável do modelo será a dependente, o ~ 1 define um intercepto (constante), as demais variáveis são as independentes; e
# EntityEffects é um instrução do pacote para incluir efeitos fixos de entidade no modelo.
mod_ef = PanelOLS.from_formula('maes_fam ~ 1 + analf + theil + gini + idhm + idhm_educ + ensino_medio + renda + EntityEffects', data=dados, check_rank=False)

# .fit realiza o ajuste do modelo aos dados.
res_ef = mod_ef.fit()

# Print do texto e print do sumário rodando o res_ef.
print("\nResultados do modelo de Efeitos Fixos:")
print(res_ef.summary)


Resultados do modelo de Efeitos Fixos:
                          PanelOLS Estimation Summary                           
Dep. Variable:               maes_fam   R-squared:                        0.7029
Estimator:                   PanelOLS   R-squared (Between):              0.6392
No. Observations:                 204   R-squared (Within):               0.7029
Date:                Fri, May 31 2024   R-squared (Overall):              0.6775
Time:                        16:26:09   Log-likelihood                   -604.36
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      32.115
Entities:                         102   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                    F(7,95)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):        

## Modelo de efeitos aleatórios
### Permite que os efeitos individuais sejam aleatórios, o que significa que eles são amostrados de uma distribuição específica.

In [ ]:
# Funções com a mesma lógica das realizadas no modelo anterior, mudando apenas o "RandomEffects", efeitos aleatórios.
mod_ea = RandomEffects.from_formula('maes_fam ~ 1 + analf + theil + gini + idhm + idhm_educ + ensino_medio + renda', data=dados)
res_ea = mod_ea.fit()
print("\nResultados do modelo de Efeitos Aleatórios:")
print(res_ea.summary)


Resultados do modelo de Efeitos Aleatórios:
                        RandomEffects Estimation Summary                        
Dep. Variable:               maes_fam   R-squared:                        0.6903
Estimator:              RandomEffects   R-squared (Between):              0.6786
No. Observations:                 204   R-squared (Within):               0.6981
Date:                Fri, May 31 2024   R-squared (Overall):              0.6903
Time:                        16:26:14   Log-likelihood                   -660.51
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      62.415
Entities:                         102   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                   F(7,196)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):   

# **Modelos dinâmicos**
### Capturam a dependência temporal e os efeitos persistentes, modelando a dinâmica das variáveis ao longo do tempo.


### GMM pode ser utilizado quando se deseja estimar um modelo parcialmente especificado.

In [ ]:
# Modelos Dinâmicos usando GMM

# Retirar novamente os valores nulos, por precaução.
dados.dropna(inplace=True)

# A operação shift(1) cria uma versão defasada da variável maes_fam, chamada maes_fam_l1
# Com objetivo de capturar dependências temporais, modelar persistência e efeitos dinâmicos.
dados['maes_fam_l1'] = dados['maes_fam'].shift(1)

# Definindo as variáveis e rodando o modelo com a mesma lógica realizada anteriormente.
formula = 'maes_fam ~ 1 + maes_fam_l1 + analf + theil + gini + idhm + idhm_educ + ensino_medio + renda'
mod_gmm = IV2SLS.from_formula(formula, data=dados)
res_gmm = mod_gmm.fit()
print("\nResultados do modelo Dinâmico usando GMM:")
print(res_gmm.summary)


Resultados do modelo Dinâmico usando GMM:
                            OLS Estimation Summary                            
Dep. Variable:               maes_fam   R-squared:                      0.7035
Estimator:                        OLS   Adj. R-squared:                 0.6912
No. Observations:                 203   F-statistic:                    678.30
Date:                Fri, May 31 2024   P-value (F-stat)                0.0000
Time:                        16:26:19   Distribution:                  chi2(8)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
Intercept        99.039     16.689     5.9344     0.0000      66.330      131.75
m

/usr/local/lib/python3.10/dist-packages/linearmodels/iv/model.py:550: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


# **Teste de Hausman**
### Verifica se os coeficientes dos dois modelos se diferem de forma significativa. Se eles forem significativamente diferentes, devemos escolher o modelo de efeitos fixos.

In [ ]:
# Compare é a função da biblioteca que faz a comparação dos dois modelos rodados anteriormente.
comparison = compare({'Efeitos Fixos': res_ef, 'Efeitos Aleatórios': res_ea})
print("\nTeste de Hausman:")
print(comparison)


Teste de Hausman:
                     Model Comparison                    
                         Efeitos Fixos Efeitos Aleatórios
---------------------------------------------------------
Dep. Variable                 maes_fam           maes_fam
Estimator                     PanelOLS      RandomEffects
No. Observations                   204                204
Cov. Est.                   Unadjusted         Unadjusted
R-squared                       0.7029             0.6903
R-Squared (Within)              0.7029             0.6981
R-Squared (Between)             0.6392             0.6786
R-Squared (Overall)             0.6775             0.6903
F-statistic                     32.115             62.415
P-value (F-stat)                0.0000             0.0000
=====================     ============    ===============
Intercept                       112.25             102.93
                              (2.8138)           (6.0758)
analf                          -0.1092             0.